<a href="https://colab.research.google.com/github/bpratham2001/Servilia/blob/main/notebooks/Wine_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
#!pip install kagglehub[pandas-datasets]
!pip install opendatasets

In [29]:
import opendatasets as od
import pandas as pd
import numpy as np
flag = False
#checked
#mysarahmadbhat/wine-tasting // winemag-data-130k-v2.csv \\ zynicide/wine-reviews is better

#potential sources
# https://en.wikipedia.org/wiki/Wine_tasting_descriptors
# https://www.wineenthusiast.com/toplists/wine/
# https://en.wikipedia.org/wiki/Wine_tasting

od.download("https://www.kaggle.com/datasets/zynicide/wine-reviews", force=flag) # winemag-data_first150k.csv, winemag-data-130k-v2.csv
file =('wine-reviews/winemag-data_first150k.csv')
df1 = pd.read_csv(file)
file =('wine-reviews/winemag-data-130k-v2.csv')
df2 = pd.read_csv(file)

od.download("https://www.kaggle.com/datasets/fedesoriano/spanish-wine-quality-dataset", force=flag) # wines_SPA.csv
file =('spanish-wine-quality-dataset/wines_SPA.csv')
df3 = pd.read_csv(file)

od.download("https://www.kaggle.com/datasets/elvinrustam/wine-dataset", force=flag) # WineDataset.csv
file =('wine-dataset/WineDataset.csv')
df4 = pd.read_csv(file)

file =('wine-tastings-guide.txt') # https://www.wine-tastings-guide.com/wine-descriptions.html
df5 = pd.read_csv(file, sep = ' - ')
file =('grapes.txt')
df6 = pd.read_csv(file, sep = ' - ')
file =('FR-IT.txt')
df6 = pd.read_csv(file, sep = ' - ')

Dataset URL: https://www.kaggle.com/datasets/zynicide/wine-reviews


100%|██████████| 50.9M/50.9M [00:02<00:00, 20.3MB/s]



Dataset URL: https://www.kaggle.com/datasets/fedesoriano/spanish-wine-quality-dataset


100%|██████████| 46.4k/46.4k [00:00<00:00, 1.56MB/s]


Dataset URL: https://www.kaggle.com/datasets/elvinrustam/wine-dataset


100%|██████████| 308k/308k [00:00<00:00, 648kB/s]


<ipython-input-29-1f4357c44fed>:28: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df5 = pd.read_csv(file, sep = ' - ')


In [12]:
df1.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
df2.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
#df['DataFrame Column'] = df['DataFrame Column'].fillna(0)

# fix designation and regions in df1 and df2 // not fixable?
#print(df1[df1['designation'].isnull()])
#print(df2[df2['designation'].isnull()])
region = np.concatenate((df1[df1['region_1'].isnull()].winery.unique(), df2[df2['region_1'].isnull()].winery.unique()))
region = np.concatenate((region, df1[df1['region_2'].isnull()].winery.unique()))
region = set(np.concatenate((region, df2[df2['region_2'].isnull()].winery.unique())))
print(str(len(region)) + " wineries of unknown region \n")
#df1.loc[(df1.region_2.isnull()) & (df1.region_1.isnull())])

# fix countries in df1 and df2, fix variety in df2
countries = {'Tsililis':'Greece', 'Büyülübağ':'Turkey', 'Chilcas':'Chile',
             'Gotsa Family Wines':'Georgia', 'Barton & Guestier':'France',
             'Kakhetia Traditional Winemaking':'Georgia', 'Ross-idi':'Bulgaria',
             'Orbelus':'Bulgaria', 'St. Donat':'Hungary', 'Familia Deicas':'Uruguay','Bartho Eksteen':'South Africa',
             'Stone Castle':'Kosovo', 'Teliani Valley':'Georgia', 'Undurraga':'Chile', 'Mt. Beautiful':'New Zealand',
             'Neumeister':'Austria', 'Santa Ema':'Chile', 'Psagot':'Israel',
             'Newton Johnson':'South Africa', 'Ktima Voyatzi':'Greece', 'Lismore':'South Africa', 'Logodaj':'Bulgaria',
             'Somlói Vándor':'Hungary', 'Les Frères Dutruy':'Switzerland', 'El Capricho':'Uruguay',
             'Amiran Vepkhvadze':'Georgia'}

df2.at[38898, 'country'] = 'Canada'
df2.at[38898, 'region_1'] = 'Twenty Mile Bench'
df2.at[38898, 'region_2'] = 'Niagara Peninsula'
df2.at[86909, 'variety'] = 'Petite Syrah'
df2.at[86909, 'region_1'] = 'Maipo Valley'

for index, row in df1.iterrows():
    if df1.at[index, 'winery'] in countries:
      df1.at[index, 'country'] = countries[df1.at[index, 'winery']]
for index, row in df2.iterrows():
    if df2.at[index, 'winery'] in countries:
      df2.at[index, 'country'] = countries[df2.at[index, 'winery']]

# merge regions in df1 and df2, drop useless columns
for i, row in df1.iterrows():
  if df1.at[i, 'region_1'] != df1.at[i, 'region_2']:
    if str(df1.at[i, 'region_1']) == 'nan':
      df1.at[i, 'region_1'] = str(df1.at[i, 'region_2'])
    elif str(df1.at[i, 'region_2']) != 'nan':
      df1.at[i, 'region_1'] = (str(df1.at[i, 'region_1']) +", "+ str(df1.at[i, 'region_2']))
df1 = df1.drop('region_2', axis=1)
df1 = df1.drop('price', axis=1)
df1 = df1.drop('points', axis=1)

for i, row in df2.iterrows():
  if df2.at[i, 'region_1'] != df2.at[i, 'region_2']:
    if str(df2.at[i, 'region_1']) == 'nan':
      df2.at[i, 'region_1'] = str(df2.at[i, 'region_2'])
    elif str(df2.at[i, 'region_2']) != 'nan':
      df2.at[i, 'region_1'] = (str(df2.at[i, 'region_1']) +", "+ str(df2.at[i, 'region_2']))
df2 = df2.drop('region_2', axis=1)
df2 = df2.drop('taster_twitter_handle', axis=1)
df2 = df2.drop('price', axis=1)

# check df1/df2 stats
for i in df1.keys():
  print(i +" "+ str(df1[i].isnull().values.any()))
df1.isnull().values.any()
print("\n")
for i in df2.keys():
  print(i +" "+ str(df2[i].isnull().values.any()))

14017
id False
country False
description False
designation True
province True
region_1 False
variety False
winery False


id False
country False
description False
designation True
points False
province True
region_1 False
taster_name True
title False
variety False
winery False


In [19]:
for i, row in df3.iterrows():
  df3.at[i, 'country'] = "Spain"
df3 = df3.drop('price', axis=1)
for i in df3.keys():
  print(i +" "+ str(df3[i].isnull().values.any()))

winery False
wine False
year True
rating False
num_reviews False
country False
region False
price False
type True
body True
acidity True


In [22]:
for i, row in df4.iterrows():
  if df4.at[i, 'Country'] == 'USA':
    df4.at[i, 'Country'] = 'US'
df4 = df4.drop('Price', axis=1)
df4 = df4.drop('Capacity', axis=1)
df4 = df4.drop('Unit', axis=1)
df4 = df4.drop('Per bottle / case / each', axis=1)
df4 = df4.drop('ABV', axis=1)
for i in df4.keys():
  print(i +" "+ str(df4[i].isnull().values.any()))

Title False
Description True
Grape True
Secondary Grape Varieties True
Closure True
Country True
Characteristics True
Type True
Region True
Style True
Vintage True
Appellation True


In [31]:
df5.head()

,term,description
0,Austere,"Austere means ""severe or strict in manner, att..."
1,Bouquet,The perfume of a wine. A wine's bouquet is gen...
2,Cassis,One of the most common wine descriptions in ta...
3,Complex,A complex wine is one which has a plethora of ...
4,Concentrated,A concentrated wine is one which is richly fla...
